In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, CategoricalCrossentropy

# Configure TensorFlow to allow GPU growth
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset_dir = "UTKFace"

# Set the batch size
batch_size = 64

#preprocessing

images = []
ages = []
genders = []
count =0
for filename in os.listdir(dataset_dir):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(dataset_dir, filename))
        img_array = np.array(img)
        images.append(img_array)
        age, gender, *_ = filename.split("_")
        ages.append(int(age))
        genders.append(int(gender))
print(images)
# Convert lists to numpy arrays
images = np.array(images)
ages = np.array(ages)
genders = to_categorical(np.array(genders))  # One-hot encode the gender labels

# Split the dataset into training and validation sets
train_images, val_images, train_ages, val_ages, train_genders, val_genders = train_test_split(
    images, ages, genders, test_size=0.2, random_state=42, stratify=genders
)
print("done")

In [ ]:
# Loading the pre-trained ResNet-50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False)

# Freezeing to keep weights
for layer in base_model.layers:
    layer.trainable = False

# Adding our layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
age_pred = Dense(1, activation='linear', name='age_output')(x)
gender_pred = Dense(2, activation='sigmoid', name='gender_output')(x)

model.compile(optimizer=Adam(),
              loss={'age_output': MeanSquaredError(), 'gender_output': CategoricalCrossentropyCrossentropy()},
              metrics={'age_output': 'mae', 'gender_output': 'accuracy'})

# Fine-tune the model
model.fit(
    train_images, {'age_output': train_ages, 'gender_output': train_genders},
    validation_data=(val_images, {'age_output': val_ages, 'gender_output': val_genders}),
    batch_size=batch_size, epochs=30)

# Save the fine-tuned model
model.save('model.h5')
